In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [2]:
# Load data from https://www.kaggle.com/datasets/rashikrahmanpritom/heart-attack-analysis-prediction-dataset?resource=download&select=heart.csv
df = pd.read_csv('heart.csv')
df.describe()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [3]:
# One hot encoding (for linear classifier)
df = pd.get_dummies(df, columns=['caa', 'cp', 'restecg'])

# Get targets
y_all = df['output'].to_numpy()

# Standardize the rest
df = (df - df.mean()) / df.std()

df.describe()

,age,sex,trtbps,chol,fbs,thalachh,exng,oldpeak,slp,thall,...,caa_2,caa_3,caa_4,cp_0,cp_1,cp_2,cp_3,restecg_0,restecg_1,restecg_2
count,3.030000e+02,3.030000e+02,3.030000e+02,3.030000e+02,3.030000e+02,3.030000e+02,3.030000e+02,3.030000e+02,3.030000e+02,3.030000e+02,...,303.000000,3.030000e+02,3.030000e+02,3.030000e+02,3.030000e+02,3.030000e+02,3.030000e+02,3.030000e+02,3.030000e+02,3.030000e+02
mean,7.035077e-17,-4.690051e-17,-6.683323e-16,-8.207589e-17,8.207589e-17,-5.862564e-16,-2.345026e-17,-4.690051e-17,-7.035077e-17,-1.876020e-16,...,0.000000,9.380102e-17,7.035077e-17,-7.035077e-17,-5.862564e-17,-2.345026e-17,-5.276307e-17,4.690051e-17,1.055261e-16,-2.345026e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.793003e+00,-1.465992e+00,-2.145254e+00,-2.320322e+00,-4.169448e-01,-3.433587e+00,-6.954800e-01,-8.953805e-01,-2.270822e+00,-3.778573e+00,...,-0.378052,-2.654018e-01,-1.293180e-01,-9.438222e-01,-4.438200e-01,-6.335996e-01,-2.861324e-01,-9.691222e-01,-1.001649e+00,-1.154720e-01
25%,-7.560295e-01,-1.465992e+00,-6.627704e-01,-6.803688e-01,-4.169448e-01,-7.049444e-01,-6.954800e-01,-8.953805e-01,-6.480412e-01,-5.120748e-01,...,-0.378052,-2.654018e-01,-1.293180e-01,-9.438222e-01,-4.438200e-01,-6.335996e-01,-2.861324e-01,-9.691222e-01,-1.001649e+00,-1.154720e-01
50%,6.977057e-02,6.798805e-01,-9.258463e-02,-1.208554e-01,-4.169448e-01,1.463921e-01,-6.954800e-01,-2.063639e-01,-6.480412e-01,-5.120748e-01,...,-0.378052,-2.654018e-01,-1.293180e-01,-9.438222e-01,-4.438200e-01,-6.335996e-01,-2.861324e-01,-9.691222e-01,9.950590e-01,-1.154720e-01
75%,7.304107e-01,6.798805e-01,4.776012e-01,5.447726e-01,-4.169448e-01,7.139498e-01,1.433110e+00,4.826527e-01,9.747397e-01,1.121174e+00,...,-0.378052,-2.654018e-01,-1.293180e-01,1.056025e+00,-4.438200e-01,1.573075e+00,-2.861324e-01,1.028456e+00,9.950590e-01,-1.154720e-01
max,2.492118e+00,6.798805e-01,3.898716e+00,6.130260e+00,2.390484e+00,2.285648e+00,1.433110e+00,4.444498e+00,9.747397e-01,1.121174e+00,...,2.636412,3.755436e+00,7.707354e+00,1.056025e+00,2.245729e+00,1.573075e+00,3.483351e+00,1.028456e+00,9.950590e-01,8.631529e+00


In [4]:
x_all = df.drop(columns=['output']).to_numpy()

# Split data to train and test
x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, random_state=42)

In [5]:
# Train basic models with default parameters
models = [LogisticRegression(max_iter=5000), RandomForestClassifier(), SVC(), GaussianProcessClassifier(), KNeighborsClassifier()]

for model in models:
    model.fit(x_train, y_train)
    print(model.__class__.__name__, model.score(x_test, y_test))

LogisticRegression 0.8524590163934426
RandomForestClassifier 0.8688524590163934
SVC 0.819672131147541
GaussianProcessClassifier 0.819672131147541
KNeighborsClassifier 0.8360655737704918
